In [1]:
import wandb

import pandas as pd
import numpy as np
import sklearn
import flwr
import xgboost as xgb
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder, StandardScaler

import missingno as mn

import platform
import sys
import subprocess
import psutil

In [2]:
print(f"Operating System: {platform.system()} {platform.release()}")
print(f"Python Version: {sys.version}")

print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")

ram = psutil.virtual_memory().total / (1024 ** 2)  # Bytes -> MB
print(f"RAM: {ram:.2f} MB")

print(f"Scikit-Learn Version: {sklearn.__version__}")
print(f"NumPy Version: {np.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"WandB Version: {wandb.__version__}")
print(f"Flower Version: {flwr.__version__}")
print(f"XGBoost Version: {xgb.__version__}")

Operating System: Darwin 24.2.0
Python Version: 3.9.21 | packaged by conda-forge | (main, Dec  5 2024, 13:47:18) 
[Clang 18.1.8 ]
Machine: arm64
Processor: arm
RAM: 16384.00 MB
Scikit-Learn Version: 1.6.1
NumPy Version: 2.0.2
Pandas Version: 2.2.3
WandB Version: 0.19.4
Flower Version: 1.14.0
XGBoost Version: 2.1.3


# Data Exploration and Preprocessing

In [3]:
columns = [
    'institute', 'age', 'workclass', 'education', 'education-num', 
    'marital-status', 'occupation', 'relationship', 'race', 
    'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 
    'native-country', 'income'
]

bankA_df = pd.read_csv("BankA.csv")
bankB_df = pd.read_csv("BankB.csv")
bankC_df = pd.read_csv("BankC.csv")

bankA_df.columns = columns
bankB_df.columns = columns
bankC_df.columns = columns

#print(bankA_df.info())
#print(bankB_df.info())
#print(bankC_df.info())


In [4]:
# Strip whitespace from string columns
for col in bankA_df.select_dtypes(include=['object']).columns:
    bankA_df[col] = bankA_df[col].str.strip()
    
for col in bankB_df.select_dtypes(include=['object']).columns:
    bankB_df[col] = bankB_df[col].str.strip()

for col in bankC_df.select_dtypes(include=['object']).columns:
    bankC_df[col] = bankC_df[col].str.strip()


# Encode target variable with explicit type conversion
bankA_df['income'] = pd.to_numeric(
    bankA_df['income'].replace({'<=50K': 0, '>50K': 1}),
    errors='coerce',
    downcast='integer'
)

bankB_df['income'] = pd.to_numeric(
    bankB_df['income'].replace({'<=50K': 0, '>50K': 1}),
    errors='coerce',
    downcast='integer'
)

bankC_df['income'] = pd.to_numeric(
    bankC_df['income'].replace({'<=50K': 0, '>50K': 1}),
    errors='coerce',
    downcast='integer'
)


/var/folders/kl/qq5gxjpd5j792x5gxwz99jrw0000gn/T/ipykernel_1946/544044200.py:14: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bankA_df['income'].replace({'<=50K': 0, '>50K': 1}),
/var/folders/kl/qq5gxjpd5j792x5gxwz99jrw0000gn/T/ipykernel_1946/544044200.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  bankB_df['income'].replace({'<=50K': 0, '>50K': 1}),
/var/folders/kl/qq5gxjpd5j792x5gxwz99jrw0000gn/T/ipykernel_1946/544044200.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future 

In [5]:
bankA_df.head(5)

,institute,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,Bank A,39,Private,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,7298,0,50,United-States,1
1,Bank A,42,Private,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,55,United-States,1
2,Bank A,34,Self-emp-not-inc,Masters,14,Married-civ-spouse,Sales,Husband,White,Male,0,0,45,United-States,1
3,Bank A,74,Federal-gov,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,2217,38,United-States,1
4,Bank A,48,Private,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,0


In [6]:
# Combine train and test datasets for preprocessing
# df_combined = pd.concat([data_train, data_test], ignore_index=True)
bankA_df.replace("?", np.nan, inplace=True)
print(f"Shape: {bankA_df.shape}")

bankB_df.replace("?", np.nan, inplace=True)
print(f"Shape: {bankB_df.shape}")

bankC_df.replace("?", np.nan, inplace=True)
print(f"Shape: {bankC_df.shape}")


Shape: (226456, 15)
Shape: (402923, 15)
Shape: (170620, 15)


In [ ]:
bankA_df.head(5)

In [ ]:
mn.matrix(bankA_df,figsize=(10,8));

In [ ]:
mn.matrix(bankB_df,figsize=(10,8));

In [ ]:
mn.matrix(bankC_df,figsize=(10,8));

In [7]:
# Identify categorical columns for future OHE and ordinal for scaling

categoricalA_columns = ['institute', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
ordinalA_columns = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

categoricalB_columns = ['institute', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
ordinalB_columns = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

categoricalC_columns = ['institute', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
ordinalC_columns = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']


In [ ]:
categoricalA_columns

In [ ]:
ordinalA_columns

In [ ]:
bankA_df.info()

In [8]:
# Identify attributes with missing values 
nan_Acolumns = bankA_df.columns[bankA_df.isna().any()].tolist()
categoricalA_columns_without_nan = [i for i in categoricalA_columns if i not in nan_Acolumns]

nan_Bcolumns = bankB_df.columns[bankB_df.isna().any()].tolist()
categoricalB_columns_without_nan = [i for i in categoricalB_columns if i not in nan_Bcolumns]

nan_Ccolumns = bankC_df.columns[bankC_df.isna().any()].tolist()
categoricalC_columns_without_nan = [i for i in categoricalC_columns if i not in nan_Ccolumns]


In [ ]:
nan_Acolumns

In [9]:
# OHE categrical attributes without missing values
bankA_df = pd.get_dummies(bankA_df, columns=categoricalA_columns_without_nan, drop_first=True)

bankB_df = pd.get_dummies(bankB_df, columns=categoricalB_columns_without_nan, drop_first=True)

bankC_df = pd.get_dummies(bankC_df, columns=categoricalC_columns_without_nan, drop_first=True)

In [ ]:
bankA_df.info()

In [ ]:
bankA_df.head()

In [10]:
# Label encode categorical attributes with missing values to prepare for imputation
le_decodeA = {}
for column in nan_Acolumns:
    leA = LabelEncoder()
    leA.fit(bankA_df[column])
    bankA_df[column] = leA.transform(bankA_df[column])
    le_decodeA[column] = leA

    # Transform missing values back to NaN after label encoding
    if np.nan in list(leA.classes_):
        nan_Alabel = list(leA.classes_).index(np.nan)
    bankA_df[column] = bankA_df[column].replace(nan_Alabel, np.nan)
 

In [ ]:
bankA_df.info()

In [11]:
le_decodeB = {}
for column in nan_Bcolumns:
    leB = LabelEncoder()
    leB.fit(bankB_df[column])
    bankB_df[column] = leB.transform(bankB_df[column])
    le_decodeB[column] = leB

    # Transform missing values back to NaN after label encoding
    if np.nan in list(leB.classes_):
        nan_Blabel = list(leB.classes_).index(np.nan)
    bankB_df[column] = bankB_df[column].replace(nan_Blabel, np.nan)

In [12]:
le_decodeC = {}
for column in nan_Ccolumns:
    leC = LabelEncoder()
    leC.fit(bankC_df[column])
    bankC_df[column] = leC.transform(bankC_df[column])
    le_decodeC[column] = leC

    # Transform missing values back to NaN after label encoding
    if np.nan in list(leC.classes_):
        nan_Clabel = list(leC.classes_).index(np.nan)
    bankC_df[column] = bankC_df[column].replace(nan_Clabel, np.nan)

In [13]:
def impute_nan_mode_multiple(df, columns):
    for column in columns:
        mode = df[column].mode()[0]
        df[column] = df[column].fillna(mode)
    return df

columns_to_impute = ['workclass', 'occupation', 'native-country']
df_imputedA = impute_nan_mode_multiple(bankA_df, columns_to_impute)

columns_to_impute = ['workclass', 'occupation', 'native-country']
df_imputedB = impute_nan_mode_multiple(bankB_df, columns_to_impute)

columns_to_impute = ['workclass', 'occupation', 'native-country']
df_imputedC = impute_nan_mode_multiple(bankC_df, columns_to_impute)

In [ ]:
nan_Acolumns

In [14]:
df_imputedA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226456 entries, 0 to 226455
Data columns (total 40 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   age                                   226456 non-null  int64  
 1   workclass                             226456 non-null  float64
 2   education-num                         226456 non-null  int64  
 3   occupation                            226456 non-null  float64
 4   capital-gain                          226456 non-null  int64  
 5   capital-loss                          226456 non-null  int64  
 6   hours-per-week                        226456 non-null  int64  
 7   native-country                        226456 non-null  float64
 8   income                                226456 non-null  int8   
 9   education_11th                        226456 non-null  bool   
 10  education_12th                        226456 non-null  bool   
 11  

In [15]:
# Transform label-encoded attributes with imputed values back to the original categorical values
for column in nan_Acolumns:
    df_imputedA[column] = le_decodeA[column].inverse_transform(df_imputedA[column].astype('int32'))

# OHE the attributes with categorical data and missing values which were imputed
df_combinedA_encoded = pd.get_dummies(df_imputedA, columns=nan_Acolumns, drop_first=True)

In [ ]:
df_combinedA_encoded.info()

In [16]:
# Transform label-encoded attributes with imputed values back to the original categorical values
for column in nan_Bcolumns:
    df_imputedB[column] = le_decodeB[column].inverse_transform(df_imputedB[column].astype('int32'))

# OHE the attributes with categorical data and missing values which were imputed
df_combinedB_encoded = pd.get_dummies(df_imputedB, columns=nan_Bcolumns, drop_first=True)

In [ ]:
df_combinedA_encoded.info()

In [17]:
# Transform label-encoded attributes with imputed values back to the original categorical values
for column in nan_Ccolumns:
    df_imputedC[column] = le_decodeC[column].inverse_transform(df_imputedC[column].astype('int32'))

# OHE the attributes with categorical data and missing values which were imputed
df_combinedC_encoded = pd.get_dummies(df_imputedC, columns=nan_Ccolumns, drop_first=True)

In [ ]:
df_combinedC_encoded.info()

In [18]:
print(df_combinedA_encoded["income"].dtypes)



int8


In [19]:
# Scale ordinal attributes
df_combinedA_encoded[ordinalA_columns] = StandardScaler().fit_transform(df_combinedA_encoded[ordinalA_columns])
df_combinedA_encoded = df_combinedA_encoded.astype(float) # set the values of the presprocessed dataset to one type
df_combinedA_encoded.shape

(226456, 97)

In [20]:
df_combinedA_encoded.head(10)

,age,education-num,capital-gain,capital-loss,hours-per-week,income,education_11th,education_12th,education_1st-4th,education_5th-6th,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,-0.081864,0.464501,0.463478,-0.253752,0.61422,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.137348,0.464501,-0.185879,-0.253752,1.00237,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-0.447216,0.873236,-0.185879,-0.253752,0.22607,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,2.475604,0.464501,-0.185879,4.542387,-0.31734,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.575771,0.873236,-0.185879,-0.253752,0.61422,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,0.941123,0.873236,-0.185879,-0.253752,0.61422,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.283489,0.464501,-0.185879,-0.253752,0.22607,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7,1.891040,0.464501,1.150918,-0.253752,-0.16208,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8,1.379546,0.873236,0.133372,-0.253752,-0.16208,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,-1.324063,0.464501,-0.185879,-0.253752,-0.93838,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [21]:
# Scale ordinal attributes
df_combinedB_encoded[ordinalB_columns] = StandardScaler().fit_transform(df_combinedB_encoded[ordinalB_columns])
df_combinedB_encoded = df_combinedB_encoded.astype(float) # set the values of the presprocessed dataset to one type
df_combinedB_encoded.shape

(402923, 97)

In [22]:
# Scale ordinal attributes
df_combinedC_encoded[ordinalC_columns] = StandardScaler().fit_transform(df_combinedC_encoded[ordinalC_columns])
df_combinedC_encoded = df_combinedC_encoded.astype(float) # set the values of the presprocessed dataset to one type
df_combinedC_encoded.shape

(170620, 97)

In [23]:
df_combinedA_encoded.to_csv('BankA_Clean.csv', index=False)

df_combinedB_encoded.to_csv('BankB_Clean.csv', index=False)

df_combinedC_encoded.to_csv('BankC_Clean.csv', index=False)
